In [14]:
import imageio
import numpy as np

# Dev stuff

In [8]:
samples = {
    "small" : "tests/small.bmp",
    "small-alpha" : "tests/small-alpha.bmp",
    "shadow" : "tests/proprietary/warhammer_map_1_1_shadow.bmp",
    "vortex" : "tests/proprietary/wh2_main_great_vortex_map_6.bmp"
}

# Input
Write your inputs here:

In [16]:
source = samples["small-alpha"]
relateDiagonals = True # Default is True
print(source)

tests/small-alpha.bmp


# Process
(Output is after this section)

In [17]:
image = imageio.imread(source)
print(image.shape)

(8, 8, 4)
